In [ ]:
import pandas as pd
import requests
import json

In [ ]:
# set client credentials obtained from snowflake security integration. issue describe security integration <integration object name>
token_url = 'https://iab50696.us-east-1.snowflakecomputing.com/oauth/token-request'
auth_url = 'https://iab50696.us-east-1.snowflakecomputing.com/oauth/authorize',
client_id = 'm9OK93k16k7Dj53F7hq5kGbA5JY='
client_secret = '3Gf38hcimIk5rrvv0CfGbQSM1xNwoXEHbXGXqqX9GKg='
redirect_uri = 'https://localhost.com'
sql_end_point = 'https://iab50696.us-east-1.snowflakecomputing.com/api/v2/statements/'

In [ ]:
# get authorization code by login in 
https://iab50696.us-east-1.snowflakecomputing.com/oauth/authorize?response_type=code&client_id=m9OK93k16k7Dj53F7hq5kGbA5JY%3D&redirect_uri=https%3A%2F%2Flocalhost.com

In [ ]:
# get refresh token
auth_code = '16D425780AEF7FA32CCB9983183D0911134F5ED2'
request_header = {'grant_type'  :   'authorization_code',
                  'client_id'   :   f'{client_id}',
                  'client_secret'   :   f'{client_secret}',
                  'redirect_uri': f'{redirect_uri}',
                  'code':   f'{auth_code}'}
response = requests.post(token_url,data=request_header)
jop = json.loads(response.text)
refresh_token = jop['refresh_token']

In [ ]:
jop['refresh_token']

In [ ]:
# get access token
request_header = {'grant_type'  :   'refresh_token',
                  'client_id'   :   f'{client_id}',
                  'client_secret'   :   f'{client_secret}',
                  'refresh_token' :  f'{refresh_token}',
                  'redirect_uri': f'{redirect_uri}'
                  }
response = requests.post(token_url,data=request_header)
jop = json.loads(response.text)
access_token = jop['access_token']

In [ ]:
print(access_token)

In [ ]:
sql =  "Select * from orders where o_orderkey in (:p_order)"
print(type(response))
print(type(response.json()))
#print(response.json())
#jd1 = json.dumps(response.json())  --convert to json string


In [ ]:
# post sql query and get sql response

payload = json.dumps({
  "statement": "Select * from orders limit 20",
  "timeout": 60,
  "database": "SNOWFLAKE_SAMPLE_DATA",
  "schema": "TPCH_SF1",
  "warehouse": "API_WH",
  "role": "PROD_API",
  "bindings": {
    "p_order": {
      "type": "TEXT",
      "value": [
        "300032"
      ]
    }
  }
})
headers = {
  'Content-Type': 'application/json',
  'Authorization':  f'Bearer {access_token}'
}

response = requests.request("POST", sql_end_point, headers=headers, data=payload)


In [87]:
# snowflake response contains metdata as list of dictionary objects and rows as list of lists. convert response to dict and create dataframe
# get column Headers
dd1 = response.json() 
df2 = pd.DataFrame(dd1['resultSetMetaData']['rowType'])
cols = df2['name'].to_list()
# get table output 
df1 = pd.DataFrame(dd1['data'],columns=cols)
df1.head(20)


,O_ORDERKEY,O_CUSTKEY,O_ORDERSTATUS,O_TOTALPRICE,O_ORDERDATE,O_ORDERPRIORITY,O_CLERK,O_SHIPPRIORITY,O_COMMENT
0,5400001,38449,F,270576.60,8147,2-HIGH,Clerk#000000540,0,times at the pending requests. pending realms ...
1,5400002,133681,F,216696.22,8157,4-NOT SPECIFIED,Clerk#000000154,0,"lyly against the pending, special requests. si..."
2,5400003,75391,O,191044.99,9407,2-HIGH,Clerk#000000388,0,"e the blithely ironic pinto beans. regular, fi"
3,5400004,46096,O,263505.65,9466,1-URGENT,Clerk#000000353,0,osits are. blithely even requests eat sometime...
4,5400005,117977,F,117459.27,8659,4-NOT SPECIFIED,Clerk#000000326,0,foxes use furiously slow asymptotes. express d...
5,5400006,126217,O,84588.56,9963,4-NOT SPECIFIED,Clerk#000000813,0,c pinto beans. blithely fin
6,5400007,15271,O,50890.64,10234,3-MEDIUM,Clerk#000000651,0,refully final theodolites cajole blithely abov...
7,5400032,416,F,24285.54,8489,5-LOW,Clerk#000000584,0,after the carefully even accounts. furiously ...
8,5400033,90325,O,181139.61,9621,4-NOT SPECIFIED,Clerk#000000154,0,ven depths haggle furiously. final account
9,5400034,62579,O,54996.61,10116,5-LOW,Clerk#000000852,0,; never special packages sleep carefully. depo...
